In [119]:
# function to simplify clauses given current assignment
# also removes satisfied clauses and remove falsified literals
def simplify(clauses, assignment):
    new_clauses = []
    for clause in clauses: # loop through clauses
        new_clause = []
        satisfied = False
        for literal in clause: # loop through literals
            var = abs(literal)
            if var in assignment:
                # if literal is satisfied by assignment, clause is also satisfied
                if (assignment[var] and literal > 0) or (not assignment[var] and literal < 0):
                    satisfied = True
                    break
                else:
                    # if code reaches here literal is false and skips literal
                    continue
            else:
                new_clause.append(literal)
        if not satisfied:
            if new_clause == []:
                new_clauses.append(new_clause)
            else:
                new_clauses.append(new_clause)
    return new_clauses

clauses = [[1, 2], [-1, 3], [-2, -3], [4, -1], [-4, 2]]
assignment = {1: True, 4: True}
new_clauses = simplify(clauses, assignment)
print(new_clauses)

[[3], [-2, -3], [2]]


In [120]:
# function to perform unit propogation step of cdcl
# returns clauses and assignments as well as any conflict clauses caused by said assignments
def unit_propagate(clauses, assignment):
    changed = True
    conflict_clause = None
    while changed:
        changed = False
        for clause in clauses:
            unassigned = [literal for literal in clause if abs(literal) not in assignment]
            # if clause is empty, conflict detected
            if len(clause) == 0:
                conflict_clause = list(assignment)[-1] # was clause instead of this
                return clauses, assignment, conflict_clause
            if len(unassigned) == 1:
                literal = unassigned[0]
                var = abs(literal)
                value = (literal > 0)
                if var in assignment and assignment[var] != value:
                    conflict_clause = clause
                    return clauses, assignment, conflict_clause
                if var not in assignment:
                    assignment[var] = value
                    clauses = simplify(clauses, assignment)
                    changed = True
                    break  # restart scanning clauses
    return clauses, assignment, conflict_clause

clauses = [[1, 4], [1, -3, -8], [1, 8, 12], 
           [2, 11], [-7, -3, 9], [-7, 8, -9], 
           [7, 8, -10], [7, 10, -12]]
assignment = {1: False, 3: True, 2: False, 7: True}
new_clauses = unit_propagate(clauses, assignment)
print(new_clauses)

([[]], {1: False, 3: True, 2: False, 7: True, 4: True, 8: False, 12: True, 11: True, 9: True}, 9)


In [121]:
# create function to look through all the clauses and add up all of the varibles that create the conflict
def create_conflict_clause(clauses, conflict_literal):
    conflict_list = []

    # loop through list of clauses
    for clause in clauses:
        # loop through literals of a single clause
        for literal in clause:
            # if a literal in the clause matches the conflict literal
            if literal == conflict_literal or literal == int(conflict_literal / -1):
                if literal > 0: # check to make sure if its the base literal or complement
                    clause.remove(conflict_literal) # remove the conflict literal from the clause
                else:
                    clause.remove(int(conflict_literal / -1)) # remove the conflict literal from the clause
                # loop through the conflict items and only add if they are not duplicates
                for literal in clause:
                        conflict_list.append(int(literal / -1))
    
    return list(set(conflict_list))

clauses = [[1, 4], [1, -3, -8], [1, 8, 12], 
           [2, 11], [-7, -3, 9], [-7, 8, -9], 
           [7, 8, -10], [7, 10, -12]]
assignment = {1: False, 3: True, 2: False, 7: True}
new_clauses = unit_propagate(clauses, assignment)
print(new_clauses[2])

# call create conflict clause
conflict_list = create_conflict_clause(clauses, new_clauses[2])
print(conflict_list)



9
[-8, 3, 7]


In [122]:
# heuristic function to pick a branching variable that is not yet assigned
def pick_branching_variable(clauses, assignment):
    for clause in clauses:
        for literal in clause:
            var = abs(literal)
            if var not in assignment:
                return var
    return None

clauses = [[1, 4], [1, -3, -8], [1, 8, 12], 
           [2, 11], [-7, -3, 9], [-7, 8, -9], 
           [7, 8, -10], [7, 10, -12]]
assignment = {1: False, 4: True}
branching_variable_choice = pick_branching_variable(clauses, assignment)
print(branching_variable_choice)

3


In [123]:
# function to analyze conflict clauses and learn a new clause from it
# gets the negation of the conflict clause and adds it to the list of clauses
# does not have non-chronological backtracking implmented yet 
# (usually you want to backtrack to where the beginning the the conflict clause occured in the search tree)
def analyze_conflict(clauses, assignment, conflict_clause):
    learned_clause = []
    for literal in conflict_clause:
        var = abs(literal)
        # negate current assignment for variable
        if var in assignment:
            learned_clause.append(-literal if assignment[var] else literal)
    backtrack_level = 0 # backtrack completely (return backtrack level 0)
    return learned_clause, backtrack_level